In [5]:
import sqlite3
import pandas as pd
import json
import binascii

In [6]:
def create_content(color_rgb):
    """
    创建CONTENT的二进制内容，color_rgb为[R,G,B]列表
    """
    # 完整的JSON结构
    style_json = {
        "type": "CIMPolygonSymbol",
        "symbolLayers": [
            {
                "type": "CIMSolidStroke",
                "enable": True,
                "capStyle": "Round",
                "joinStyle": "Round",
                "lineStyle3D": "Strip",
                "miterLimit": 4,
                "width": 0,
                "color": {
                    "type": "CIMRGBColor",
                    "values": color_rgb + [100]  # 添加透明度100
                }
            },
            {
                "type": "CIMSolidFill",
                "enable": True,
                "color": {
                    "type": "CIMRGBColor",
                    "values": color_rgb + [100]  # 添加透明度100
                }
            }
        ],
        "angleAlignment": "Map"
    }
    
    # 将JSON转换为字符串，然后转换为二进制
    json_str = json.dumps(style_json, separators=(',', ':'))  # 移除空格以匹配原格式
    return json_str.encode('utf-8') + b'\x00'  # 添加结尾的null字节

def add_style_to_stylx(stylx_path, excel_path):
    """
    从Excel读取数据并添加到stylx文件
    """
    try:
        # 连接到stylx数据库
        conn = sqlite3.connect(stylx_path)
        cursor = conn.cursor()
        
        # 读取Excel文件
        df = pd.read_excel(excel_path)
        
        # 对每行数据进行处理
        for index, row in df.iterrows():
            name = row['名称']          # Excel中的名称列
            color = row['RGB']        # Excel中的颜色列，格式为"R,G,B"
            key = str(index)            # Excel中的KEY列
            category = row['层级'] # Excel中的类别列
            # 解析颜色值
            rgb = [int(x) for x in color.split(',')]
            
            # 生成CONTENT
            content = create_content(rgb)
            if "(" in name and category == '土类':
                has_symble_name = name
                has_key = key
                no_symble_name = name.split('(')[0] + name.split(')')[1]
                no_symble_key = str(index)+'_1'
                print(has_symble_name, no_symble_name)
                # 插入数据
                cursor.execute("""
                    INSERT INTO ITEMS (CLASS, CATEGORY, NAME, TAGS, CONTENT, KEY)
                    VALUES (?, ?, ?, ?, ?, ?)
                """, (
                    5,                      # CLASS固定为5
                    "土类",                 # CATEGORY固定为"土类"
                    has_symble_name,                   # 从Excel读取的名称
                    "rgb;黑色;多图层",      # TAGS固定值
                    content,                # 生成的CONTENT
                    has_key                     # 从Excel读取的KEY
                ))
                
                # 插入数据
                cursor.execute("""
                    INSERT INTO ITEMS (CLASS, CATEGORY, NAME, TAGS, CONTENT, KEY)
                    VALUES (?, ?, ?, ?, ?, ?)
                """, (
                    5,                      # CLASS固定为5
                    "土类",                 # CATEGORY固定为"土类"
                    no_symble_name,                   # 从Excel读取的名称
                    "rgb;黑色;多图层",      # TAGS固定值
                    content,                # 生成的CONTENT
                    no_symble_key                     # 从Excel读取的KEY
                ))
                
            elif "(" in name and category == '土种':
                has_symble_name = name
                no_symble_name = name.replace('(', '').replace(')', '')
                no_symble_content_name = name.split('(')[0] + name.split(')')[1]
                has_key = key
                no_symble_key = str(index)+'_1'
                no_symble_content_key = str(index)+'_2'
                # 插入数据
                cursor.execute("""
                INSERT INTO ITEMS (CLASS, CATEGORY, NAME, TAGS, CONTENT, KEY)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (
                5,                      # CLASS固定为5
                "土种",                 # CATEGORY固定为"土种"
                has_symble_name,                   # 从Excel读取的名称
                "rgb;黑色;多图层",      # TAGS固定值
                content,                # 生成的CONTENT
                has_key                     # 从Excel读取的KEY
            ))
                
                # 插入数据
                cursor.execute("""
                INSERT INTO ITEMS (CLASS, CATEGORY, NAME, TAGS, CONTENT, KEY)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (
                5,                      # CLASS固定为5
                "土种",                 # CATEGORY固定为"土种"
                no_symble_name,                   # 从Excel读取的名称
                "rgb;黑色;多图层",      # TAGS固定值
                content,                # 生成的CONTENT
                no_symble_key                     # 从Excel读取的KEY
            ))
                # 插入数据
                cursor.execute("""
                INSERT INTO ITEMS (CLASS, CATEGORY, NAME, TAGS, CONTENT, KEY)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (
                5,                      # CLASS固定为5
                "土种",                 # CATEGORY固定为"土种"
                no_symble_content_name,                   # 从Excel读取的名称
                "rgb;黑色;多图层",      # TAGS固定值
                content,                # 生成的CONTENT
                no_symble_content_key                     # 从Excel读取的KEY
            ))
            else:
                # 插入数据
                cursor.execute("""
                INSERT INTO ITEMS (CLASS, CATEGORY, NAME, TAGS, CONTENT, KEY)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (
                5,                      # CLASS固定为5
                "土类",                 # CATEGORY固定为"土类"
                name,                   # 从Excel读取的名称
                "rgb;黑色;多图层",      # TAGS固定值
                content,                # 生成的CONTENT
                key                     # 从Excel读取的KEY
            ))
        
        # 提交更改
        conn.commit()
        print("成功添加样式")
        
    except Exception as e:
        print(f"发生错误: {str(e)}")
        conn.rollback()
    
    finally:
        conn.close()

In [ ]:
# 使用示例
if __name__ == "__main__":
    stylx_path = r"C:\Users\Runker\Desktop\gz_soiltype_style.stylx"
    excel_path = r"D:\worker\工作\work\三普\数据\色标\result_rgb.xlsx"
    add_style_to_stylx(stylx_path, excel_path)